In [4]:
import requests

def fetch_youtube_data(api_key, query, max_results=10):
    url = "https://www.googleapis.com/youtube/v3/search"
    params = {
        "part": "snippet",
        "q": query,
        "type": "video",
        "maxResults": max_results,
        "key": api_key
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        results = response.json().get("items", [])
        videos = [
            {
                "title": item["snippet"]["title"],
                "description": item["snippet"]["description"],
                "link": f"https://www.youtube.com/watch?v={item['id']['videoId']}"
            }
            for item in results
        ]
        return videos
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return []

In [6]:
from dotenv import load_dotenv
import os

In [12]:
load_dotenv()

True

In [14]:
API_KEY = os.getenv("YOUTUBE_API_KEY")
if API_KEY:
    print("API key loaded successfully!")
else:
    print("API key not found. Check your .env file.")

API key loaded successfully!


In [16]:
queries_by_category = {
    "Mindfulness and Relaxation": [
        "meditation", "guided meditation", "mindfulness meditation",
        "relaxation techniques", "breathing exercises", "stress relief exercises"
    ],
    "Cognitive Behavioral Therapy (CBT) and Self-Help": [
        "CBT techniques", "journaling prompts", "self-help strategies",
        "emotional resilience", "positive affirmations"
    ],
    "Physical and Somatic Practices": [
        "yoga for stress relief", "somatic release exercises",
        "progressive muscle relaxation", "tai chi for relaxation"
    ],
    "Productivity and Focus": [
        "focus meditation", "productivity techniques",
        "time management strategies", "deep work focus"
    ],
    "Sleep and Rest": [
        "sleep meditation", "bedtime relaxation",
        "guided meditation for sleep", "calming sounds for sleep"
    ],
    "General Well-Being": [
        "mental health tips", "well-being exercises",
        "positive mindset techniques", "building healthy habits"
    ]
}

In [18]:
import requests

def fetch_youtube_data(api_key, query, max_results=5):
    url = "https://www.googleapis.com/youtube/v3/search"
    params = {
        "part": "snippet",
        "q": query,
        "type": "video",
        "maxResults": max_results,
        "key": api_key
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        results = response.json().get("items", [])
        videos = [
            {
                "title": item["snippet"]["title"],
                "description": item["snippet"]["description"],
                "link": f"https://www.youtube.com/watch?v={item['id']['videoId']}"
            }
            for item in results
        ]
        return videos
    else:
        print(f"Error fetching data for query '{query}': {response.status_code}")
        return []


def fetch_data_for_all_categories(api_key, queries_by_category):
    all_data = {}
    for category, queries in queries_by_category.items():
        print(f"Fetching data for category: {category}")
        category_videos = []
        for query in queries:
            videos = fetch_youtube_data(api_key, query)
            category_videos.extend(videos)
        all_data[category] = category_videos
    return all_data

In [20]:
all_videos_by_category = fetch_data_for_all_categories(API_KEY, queries_by_category)

Fetching data for category: Mindfulness and Relaxation
Fetching data for category: Cognitive Behavioral Therapy (CBT) and Self-Help
Fetching data for category: Physical and Somatic Practices
Fetching data for category: Productivity and Focus
Fetching data for category: Sleep and Rest
Fetching data for category: General Well-Being


In [34]:
from fpdf import FPDF

def sanitize_text(text):
    """Remove unsupported characters for fpdf."""
    return text.encode('latin-1', 'replace').decode('latin-1')

def create_pdf_from_data(output_path, data):
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", style='B', size=16)

    # Add Title
    pdf.cell(0, 10, sanitize_text("YouTube Data Organized by Categories"), ln=True)
    pdf.ln(10)

    # Loop through each category and its videos
    for category, videos in data.items():
        pdf.set_font("Arial", style='B', size=14)
        pdf.cell(0, 10, sanitize_text(category), ln=True)  # Sanitize category names
        pdf.ln(5)

        for video in videos:
            pdf.set_font("Arial", style='B', size=12)
            pdf.cell(0, 10, sanitize_text(video["title"]), ln=True)  # Sanitize video title
            pdf.set_font("Arial", size=10)
            pdf.multi_cell(0, 10, f"Description: {sanitize_text(video['description'])}")  # Sanitize description
            pdf.ln(5)
            pdf.set_text_color(0, 0, 255)  # Blue for links
            pdf.cell(0, 10, f"Link: {video['link']}", ln=True, link=video["link"])  # Links don't need sanitization
            pdf.set_text_color(0, 0, 0)  # Reset color to black
            pdf.ln(10)

    # Save PDF
    pdf.output(output_path)
    print(f"PDF saved to {output_path}")

In [36]:
create_pdf_from_data("youtube_data.pdf", all_videos_by_category)

PDF saved to youtube_data.pdf


In [ ]:
queries = [
    "meditation", "guided meditation", "stress relief exercises",
    "yoga for stress relief", "CBT techniques", "self-help strategies",
    "focus meditation", "sleep meditation"
]

url = "https://www.googleapis.com/youtube/v3/search"

all_videos = []

for query in queries:
    print(f"Fetching videos for: {query}")
    params = {
        "part": "snippet",
        "q": query,
        "type": "video",
        "maxResults": 5,
        "key": API_KEY
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        results = response.json().get("items", [])
        videos = [
            {
                "title": item["snippet"]["title"],
                "description": item["snippet"]["description"],
                "link": f"https://www.youtube.com/watch?v={item['id']['videoId']}"
            }
            for item in results
        ]
        all_videos.extend(videos)
    else:
        print(f"Error fetching for {query}: {response.status_code} - {response.text}")

# Save all collected video data to a PDF or file
print(f"Collected {len(all_videos)} videos!")